In [1]:
#PIP INSTALLATION
print('---1---')
%pip install pandas
print('---2---')
%pip install seaborn
print('---3---')
%pip install scikit-learn
print('---4---')
%pip install catboost
print('---5---')
%pip install ipywidgets
print('---6---')
%pip install pandas openpyxl


---1---
Note: you may need to restart the kernel to use updated packages.
---2---
Note: you may need to restart the kernel to use updated packages.
---3---
Note: you may need to restart the kernel to use updated packages.
---4---
Note: you may need to restart the kernel to use updated packages.
---5---
Note: you may need to restart the kernel to use updated packages.
---6---
Note: you may need to restart the kernel to use updated packages.


In [97]:
#FROM LIBRARIES
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from catboost import CatBoostClassifier, Pool
from IPython.display import FileLink
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import classification_report
#IMPORT AS
import numpy as np
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

In [3]:
file_path = 'Dataset/diabetic_data.csv'
df = pd.read_csv(file_path,keep_default_na=False)

In [4]:
df_clean = df.copy()

In [5]:
df_clean.head()

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),?,6,25,1,1,...,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),?,1,1,7,3,...,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),?,1,1,7,2,...,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),?,1,1,7,2,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),?,1,1,7,1,...,No,Steady,No,No,No,No,No,Ch,Yes,NO


In [6]:
df_clean.describe()

,encounter_id,patient_nbr,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses
count,1.017660e+05,1.017660e+05,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000
mean,1.652016e+08,5.433040e+07,2.024006,3.715642,5.754437,4.395987,43.095641,1.339730,16.021844,0.369357,0.197836,0.635566,7.422607
std,1.026403e+08,3.869636e+07,1.445403,5.280166,4.064081,2.985108,19.674362,1.705807,8.127566,1.267265,0.930472,1.262863,1.933600
min,1.252200e+04,1.350000e+02,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000
25%,8.496119e+07,2.341322e+07,1.000000,1.000000,1.000000,2.000000,31.000000,0.000000,10.000000,0.000000,0.000000,0.000000,6.000000
50%,1.523890e+08,4.550514e+07,1.000000,1.000000,7.000000,4.000000,44.000000,1.000000,15.000000,0.000000,0.000000,0.000000,8.000000
75%,2.302709e+08,8.754595e+07,3.000000,4.000000,7.000000,6.000000,57.000000,2.000000,20.000000,0.000000,0.000000,1.000000,9.000000
max,4.438672e+08,1.895026e+08,8.000000,28.000000,25.000000,14.000000,132.000000,6.000000,81.000000,42.000000,76.000000,21.000000,16.000000


In [8]:
print(list(df_clean))

['encounter_id', 'patient_nbr', 'race', 'gender', 'age', 'weight', 'admission_type_id', 'discharge_disposition_id', 'admission_source_id', 'time_in_hospital', 'payer_code', 'medical_specialty', 'num_lab_procedures', 'num_procedures', 'num_medications', 'number_outpatient', 'number_emergency', 'number_inpatient', 'diag_1', 'diag_2', 'diag_3', 'number_diagnoses', 'max_glu_serum', 'A1Cresult', 'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide', 'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide', 'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone', 'tolazamide', 'examide', 'citoglipton', 'insulin', 'glyburide-metformin', 'glipizide-metformin', 'glimepiride-pioglitazone', 'metformin-rosiglitazone', 'metformin-pioglitazone', 'change', 'diabetesMed', 'readmitted']


Data Cleaning 

In [9]:
#REMOVING DUPLICATE DATA'S
df_clean['patient_nbr'].value_counts()

patient_nbr
88785891     40
43140906     28
1660293      23
88227540     23
23199021     23
             ..
11005362      1
98252496      1
1019673       1
13396320      1
175429310     1
Name: count, Length: 71518, dtype: int64

In [10]:
df_clean = df_clean.drop_duplicates(subset=['patient_nbr'])
#END REMOVING DUPLICATE DATA'S

There are 30,248 total of Duplicate Data's which the unique data consist only 71,518

In [11]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 71518 entries, 0 to 101765
Data columns (total 50 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   encounter_id              71518 non-null  int64 
 1   patient_nbr               71518 non-null  int64 
 2   race                      71518 non-null  object
 3   gender                    71518 non-null  object
 4   age                       71518 non-null  object
 5   weight                    71518 non-null  object
 6   admission_type_id         71518 non-null  int64 
 7   discharge_disposition_id  71518 non-null  int64 
 8   admission_source_id       71518 non-null  int64 
 9   time_in_hospital          71518 non-null  int64 
 10  payer_code                71518 non-null  object
 11  medical_specialty         71518 non-null  object
 12  num_lab_procedures        71518 non-null  int64 
 13  num_procedures            71518 non-null  int64 
 14  num_medications           

Value of each drug that most patient uses

In [13]:
print(df_clean['metformin'].value_counts())
print("-----------------------------")
print(df_clean['repaglinide'].value_counts())
print("-----------------------------")
print(df_clean['nateglinide'].value_counts())
print("-----------------------------")
print(df_clean['chlorpropamide'].value_counts())

metformin
No        56527
Steady    13718
Up          838
Down        435
Name: count, dtype: int64
-----------------------------
repaglinide
No        70592
Steady      824
Up           73
Down         29
Name: count, dtype: int64
-----------------------------
nateglinide
No        71020
Steady      474
Up           16
Down          8
Name: count, dtype: int64
-----------------------------
chlorpropamide
No        71446
Steady       67
Up            4
Down          1
Name: count, dtype: int64


In [14]:
print(df_clean['glimepiride'].value_counts())
print("-----------------------------")
print(df_clean['acetohexamide'].value_counts())
print("-----------------------------")
print(df_clean['glipizide'].value_counts())
print("-----------------------------")
print(df_clean['glyburide'].value_counts())


glimepiride
No        67773
Steady     3372
Up          235
Down        138
Name: count, dtype: int64
-----------------------------
acetohexamide
No        71517
Steady        1
Name: count, dtype: int64
-----------------------------
glipizide
No        62412
Steady     8150
Up          578
Down        378
Name: count, dtype: int64
-----------------------------
glyburide
No        63664
Steady     6812
Up          621
Down        421
Name: count, dtype: int64


In [15]:
print(df_clean['tolbutamide'].value_counts())
print("-----------------------------")
print(df_clean['pioglitazone'].value_counts())
print("-----------------------------")
print(df_clean['rosiglitazone'].value_counts())
print("-----------------------------")
print(df_clean['acarbose'].value_counts())

tolbutamide
No        71499
Steady       19
Name: count, dtype: int64
-----------------------------
pioglitazone
No        66210
Steady     5047
Up          180
Down         81
Name: count, dtype: int64
-----------------------------
rosiglitazone
No        66817
Steady     4490
Up          136
Down         75
Name: count, dtype: int64
-----------------------------
acarbose
No        71316
Steady      192
Up           10
Name: count, dtype: int64


In [16]:
print(df_clean['miglitol'].value_counts())
print("-----------------------------")
print(df_clean['troglitazone'].value_counts())
print("-----------------------------")
print(df_clean['tolazamide'].value_counts())
print("-----------------------------")
print(df_clean['examide'].value_counts())

miglitol
No        71498
Steady       18
Down          1
Up            1
Name: count, dtype: int64
-----------------------------
troglitazone
No        71515
Steady        3
Name: count, dtype: int64
-----------------------------
tolazamide
No        71488
Steady       30
Name: count, dtype: int64
-----------------------------
examide
No    71518
Name: count, dtype: int64


In [17]:
print(df_clean['citoglipton'].value_counts())
print("-----------------------------")
print(df_clean['insulin'].value_counts())
print("-----------------------------")
print(df_clean['glyburide-metformin'].value_counts())
print("-----------------------------")
print(df_clean['glipizide-metformin'].value_counts())

citoglipton
No    71518
Name: count, dtype: int64
-----------------------------
insulin
No        34921
Steady    22129
Down       7505
Up         6963
Name: count, dtype: int64
-----------------------------
glyburide-metformin
No        71016
Steady      491
Up            7
Down          4
Name: count, dtype: int64
-----------------------------
glipizide-metformin
No        71511
Steady        7
Name: count, dtype: int64


In [31]:
print(df_clean['glimepiride-pioglitazone'].value_counts())
print("-----------------------------")
print(df_clean['metformin-rosiglitazone'].value_counts())
print("-----------------------------")
print(df_clean['metformin-pioglitazone'].value_counts())

glimepiride-pioglitazone
No    71518
Name: count, dtype: int64
-----------------------------
metformin-rosiglitazone
No        71516
Steady        2
Name: count, dtype: int64
-----------------------------
metformin-pioglitazone
No        71517
Steady        1
Name: count, dtype: int64


In [30]:
(df_clean.medical_specialty.value_counts(normalize=True, dropna=False) * 100)


medical_specialty
?                                   48.207444
InternalMedicine                    15.267485
Family/GeneralPractice               7.156240
Emergency/Trauma                     6.243184
Cardiology                           5.964932
                                      ...    
SportsMedicine                       0.001398
Dermatology                          0.001398
Proctology                           0.001398
Surgery-PlasticwithinHeadandNeck     0.001398
Resident                             0.001398
Name: proportion, Length: 71, dtype: float64

Removing features that is not useful for our target readmittied

encounter_id  > Irrelevant 

Patient_nbr > Irrelevant

Weight > Consist of 97% missing value

Payer_code > Irrelevant

Medicial_Speciality > 48% missing value

Repaglinide	> 71K samples not using the drug

Nateglinide	> 71K samples not using the drug

Chlorpropamide > 71K samples not using the drug

Acarbose > 71k samples not using the drug

Miglitol > 71k samples not using the drug

Troglitazone > Only 3 patients using the drug

Tolazamide > Only 39 patients using the drug

Examide	> All patients not using the drug

Citoglipton	> All patients not using the drug

Glyburide_metformin	> 71k samples not using the drug

Glipizide_metfotmin > Only 13 patients using the drug

Glimepiride_pioglitazone > Only 1 patient using the drug

Metformin_rosiglitazone > Only 2 patients using the drug

Metformin_pioglitazone > Only 1 patient using the drug

Acetohexamide > Only 1 patient using the drug

tolbutamide > Only 23 patients using the drug



In [32]:
features_drop_list = ['encounter_id', 'patient_nbr', 'weight', 'payer_code', 'medical_specialty', 'repaglinide', 'nateglinide', 'chlorpropamide', 'acarbose', 'miglitol', 'troglitazone', 'tolazamide', 'examide', 'citoglipton', 'glyburide-metformin', 'glipizide-metformin', 'glimepiride-pioglitazone', 'metformin-rosiglitazone','metformin-pioglitazone', 'acetohexamide', 'tolbutamide']
df_clean.drop(features_drop_list, axis=1, inplace=True)

In [34]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 71518 entries, 0 to 101765
Data columns (total 29 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   race                      71518 non-null  object
 1   gender                    71518 non-null  object
 2   age                       71518 non-null  object
 3   admission_type_id         71518 non-null  int64 
 4   discharge_disposition_id  71518 non-null  int64 
 5   admission_source_id       71518 non-null  int64 
 6   time_in_hospital          71518 non-null  int64 
 7   num_lab_procedures        71518 non-null  int64 
 8   num_procedures            71518 non-null  int64 
 9   num_medications           71518 non-null  int64 
 10  number_outpatient         71518 non-null  int64 
 11  number_emergency          71518 non-null  int64 
 12  number_inpatient          71518 non-null  int64 
 13  diag_1                    71518 non-null  object
 14  diag_2                    

ICD 9 codes

In [35]:
#start by setting all values containing E or V into 0 (as one category)
df_clean.loc[df_clean['diag_1'].str.contains('V',na=False,case=False), 'diag_1'] = 0
df_clean.loc[df_clean['diag_1'].str.contains('E',na=False,case=False), 'diag_1'] = 0
df_clean.loc[df_clean['diag_2'].str.contains('V',na=False,case=False), 'diag_2'] = 0
df_clean.loc[df_clean['diag_2'].str.contains('E',na=False,case=False), 'diag_2'] = 0
df_clean.loc[df_clean['diag_3'].str.contains('V',na=False,case=False), 'diag_3'] = 0
df_clean.loc[df_clean['diag_3'].str.contains('E',na=False,case=False), 'diag_3'] = 0

#setting all missing values into -1
df_clean['diag_1'] = df_clean['diag_1'].replace('?', -1)
df_clean['diag_2'] = df_clean['diag_2'].replace('?', -1)
df_clean['diag_3'] = df_clean['diag_3'].replace('?', -1)

#No all diag values can be converted into numeric values
df_clean['diag_1'] = df_clean['diag_1'].astype(float)
df_clean['diag_2'] = df_clean['diag_2'].astype(float)
df_clean['diag_3'] = df_clean['diag_3'].astype(float)

In [37]:
# First convert diagnosis columns to numeric type
df_clean['diag_1'] = pd.to_numeric(df_clean['diag_1'], errors='coerce')
df_clean['diag_2'] = pd.to_numeric(df_clean['diag_2'], errors='coerce')
df_clean['diag_3'] = pd.to_numeric(df_clean['diag_3'], errors='coerce')

# Define a function to categorize diagnosis codes
def categorize_diagnosis(series):
    conditions = [
        (series >= 1) & (series < 140),
        (series >= 140) & (series < 240),
        (series >= 240) & (series < 280),
        (series >= 280) & (series < 290),
        (series >= 290) & (series < 320),
        (series >= 320) & (series < 390),
        (series >= 390) & (series < 460),
        (series >= 460) & (series < 520),
        (series >= 520) & (series < 580),
        (series >= 580) & (series < 630),
        (series >= 630) & (series < 680),
        (series >= 680) & (series < 710),
        (series >= 710) & (series < 740),
        (series >= 740) & (series < 760),
        (series >= 760) & (series < 780),
        (series >= 780) & (series < 800),
        (series >= 800) & (series < 1000)
    ]
    
    choices = list(range(1, 18))
    
    # Handle missing values and set them to 0
    result = np.select(conditions, choices, default=0)
    return result

# Apply categorization to each diagnosis column
df_clean['diag_1'] = categorize_diagnosis(df['diag_1'])
df_clean['diag_2'] = categorize_diagnosis(df['diag_2'])
df_clean['diag_3'] = categorize_diagnosis(df['diag_3'])

In [84]:
#check the results
df_clean.diag_1.unique()

array([250.83, 276.  , 648.  ,   8.  , 197.  , 414.  , 428.  , 398.  ,
       434.  , 250.7 , 157.  , 518.  , 999.  , 410.  , 682.  , 402.  ,
       737.  , 572.  ,   0.  , 189.  , 786.  , 427.  , 996.  , 277.  ,
       584.  , 462.  , 473.  , 411.  , 174.  , 486.  , 998.  , 511.  ,
       432.  , 626.  , 295.  , 196.  , 250.6 , 618.  , 182.  , 845.  ,
       423.  , 808.  , 250.4 , 722.  , 403.  , 250.11, 784.  , 707.  ,
       440.  , 151.  , 715.  , 997.  , 564.  , 812.  ,  38.  , 590.  ,
       556.  , 578.  , 250.32, 433.  , 569.  , 185.  , 536.  , 255.  ,
       250.13, 599.  , 558.  , 574.  , 491.  , 560.  , 244.  , 250.03,
       577.  , 730.  , 188.  , 824.  , 250.8 , 332.  , 562.  , 291.  ,
       296.  , 510.  , 401.  , 263.  , 438.  ,  70.  , 250.02, 493.  ,
       642.  , 625.  , 571.  , 738.  , 593.  , 250.42, 807.  , 456.  ,
       446.  , 575.  , 250.41, 820.  , 515.  , 780.  , 250.22, 995.  ,
       235.  , 250.82, 721.  , 787.  , 162.  , 724.  , 282.  , 514.  ,
      

In [83]:
# 1. Basic value counts to see distribution of categories
print("Distribution of diagnosis 1 categories:")
print(df_clean['diag_1'].value_counts().sort_index())


Distribution of diagnosis 1 categories:
diag_1
-1.0        11
 0.0       928
 3.0        12
 5.0         8
 7.0         2
          ... 
 995.0      52
 996.0    1125
 997.0     255
 998.0     457
 999.0      18
Name: count, Length: 680, dtype: int64


In [41]:
print("\nDistribution of diagnosis 2 categories:")
print(df['diag_2'].value_counts().sort_index())


Distribution of diagnosis 2 categories:
diag_2
0      2894
1      1931
2      2547
3     21017
4      2926
5      2657
6      1286
7     31365
8     10251
9      3962
10     7987
11      415
12     3596
13     1764
14      108
16     4632
17     2428
Name: count, dtype: int64


In [40]:
print("\nDistribution of diagnosis 3 categories:")
print(df['diag_3'].value_counts().sort_index())


Distribution of diagnosis 3 categories:
diag_3
0      6481
1      1861
2      1856
3     26308
4      2490
5      3136
6      1766
7     29918
8      6774
9      3572
10     6327
11      309
12     2488
13     1915
14       96
16     4523
17     1946
Name: count, dtype: int64


In [49]:
print(df_clean.head())

              race  gender      age  admission_type_id  \
0        Caucasian  Female   [0-10)                  6   
1        Caucasian  Female  [10-20)                  1   
2  AfricanAmerican  Female  [20-30)                  1   
3        Caucasian    Male  [30-40)                  1   
4        Caucasian    Male  [40-50)                  1   

   discharge_disposition_id  admission_source_id  time_in_hospital  \
0                        25                    1                 1   
1                         1                    7                 3   
2                         1                    7                 2   
3                         1                    7                 2   
4                         1                    7                 1   

   num_lab_procedures  num_procedures  num_medications  ...  metformin  \
0                  41               0                1  ...         No   
1                  59               0               18  ...         No   
2       

In [50]:
df_clean['race'] = df_clean['race'].replace('?', 'Other')

In [51]:
df_clean.race.value_counts()

race
Caucasian          53491
AfricanAmerican    12887
Other               3126
Hispanic            1517
Asian                497
Name: count, dtype: int64

In [52]:
print(df_clean['race'].isna())

0         False
1         False
2         False
3         False
4         False
          ...  
101754    False
101755    False
101756    False
101758    False
101765    False
Name: race, Length: 71518, dtype: bool


In [53]:
df_clean.gender.value_counts()

gender
Female             38025
Male               33490
Unknown/Invalid        3
Name: count, dtype: int64

In [54]:
df_clean['gender'] = df_clean['gender'].replace('Unknown/Invalid', 'Female')
df_clean.gender.value_counts()

gender
Female    38028
Male      33490
Name: count, dtype: int64

In [55]:
df_clean.age.value_counts()


age
[70-80)     18210
[60-70)     15960
[50-60)     12466
[80-90)     11589
[40-50)      6878
[30-40)      2699
[90-100)     1900
[20-30)      1127
[10-20)       535
[0-10)        154
Name: count, dtype: int64

In [56]:
for i in range(0, 10):
    age_range = f'[{10*i}-{10*(i+1)})'
    midpoint = i*10 + 5
    df['age'] = df['age'].replace(age_range, midpoint)

C:\Users\SnowPlay\AppData\Local\Temp\ipykernel_11628\3365448648.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['age'] = df['age'].replace(age_range, midpoint)


In [57]:
df_clean['age'].value_counts()

age
[70-80)     18210
[60-70)     15960
[50-60)     12466
[80-90)     11589
[40-50)      6878
[30-40)      2699
[90-100)     1900
[20-30)      1127
[10-20)       535
[0-10)        154
Name: count, dtype: int64

In [58]:
df_clean.max_glu_serum.value_counts()


max_glu_serum
None    68062
Norm     1731
>200      969
>300      756
Name: count, dtype: int64

In [59]:
label_encoder = LabelEncoder()
df_clean['readmitted'] = label_encoder.fit_transform(df_clean['readmitted'])

In [60]:
X = df_clean.drop('readmitted', axis=1)  # Features
y = df_clean['readmitted']               # Target variable


In [61]:
cat_features = [index for index, dtype in enumerate(X.dtypes) if dtype == 'object']


In [62]:
print(X.iloc[:, cat_features].isnull().sum())


race             0
gender           0
age              0
max_glu_serum    0
A1Cresult        0
metformin        0
glimepiride      0
glipizide        0
glyburide        0
pioglitazone     0
rosiglitazone    0
insulin          0
change           0
diabetesMed      0
dtype: int64


In [63]:
# Save as Excel
df_clean.to_excel('clean_dataframe.xlsx', index=False, engine='openpyxl')

In [64]:
FileLink('clean_dataframe.xlsx')

d:\optimizing-catboost-model-with-bayesian\clean_dataframe.xlsx

Training Model

In [90]:
X_train, X_valtest, y_train, y_valtest = train_test_split(X, y, test_size=0.3, random_state=101)
X_val, X_test, y_val, y_test = train_test_split(X_valtest, y_valtest, test_size=0.5, random_state=101)


In [92]:
train_pool = Pool(data=X_train, label=y_train, cat_features=cat_features)
val_pool = Pool(
    data=X_val,           # Feature set for validation
    label=y_val,          # Target variable for validation
    cat_features=cat_features     # Specify indices of categorical features if present
)
test_pool = Pool(data=X_test, label=y_test, cat_features=cat_features)


In [93]:
# Step 5: Train the CatBoost model
model = CatBoostClassifier(
    loss_function = 'MultiClass',
    iterations=1000,          # Number of boosting rounds
    learning_rate=0.05,       # Step size shrinkage
    depth=6,                  # Depth of trees
    verbose=100              # Display training logs
)


In [94]:
model.fit(
    X_train, y_train,
    cat_features=cat_features,
    eval_set=(X_test, y_test),

)

0:	learn: 1.0776098	test: 1.0777598	best: 1.0777598 (0)	total: 110ms	remaining: 1m 50s
50:	learn: 0.8514527	test: 0.8539671	best: 0.8539671 (50)	total: 5.31s	remaining: 1m 38s
100:	learn: 0.8365138	test: 0.8427880	best: 0.8427880 (100)	total: 10.9s	remaining: 1m 37s
150:	learn: 0.8301892	test: 0.8391821	best: 0.8391821 (150)	total: 16.5s	remaining: 1m 32s
200:	learn: 0.8261884	test: 0.8374413	best: 0.8374332 (199)	total: 22.1s	remaining: 1m 27s
250:	learn: 0.8208505	test: 0.8355392	best: 0.8355392 (250)	total: 28.1s	remaining: 1m 23s
300:	learn: 0.8152538	test: 0.8329752	best: 0.8329752 (300)	total: 33.9s	remaining: 1m 18s
350:	learn: 0.8106115	test: 0.8316798	best: 0.8316798 (350)	total: 39.8s	remaining: 1m 13s
400:	learn: 0.8068081	test: 0.8307602	best: 0.8307462 (397)	total: 45.8s	remaining: 1m 8s
450:	learn: 0.8031975	test: 0.8298277	best: 0.8298202 (449)	total: 52.4s	remaining: 1m 3s
500:	learn: 0.7997367	test: 0.8291253	best: 0.8291223 (498)	total: 58.4s	remaining: 58.2s
550:	lea

In [102]:
# Step 9: Make predictions
prediction = model.predict(X_val)

In [103]:
print('\nCatboost:')
print('-----------------------------------')
print('F1 Score        : ', metrics.f1_score(y_val, prediction,average='micro'))
print('Confusion Matrix:\n ', confusion_matrix(y_val, prediction))


Catboost:
-----------------------------------
F1 Score        :  0.6151193139448173
Confusion Matrix:
  [[  11  194  732]
 [   6  666 2718]
 [   6  473 5922]]


In [106]:
# Step 10: Evaluate the model
accuracy = accuracy_score(y_val, prediction)
print(f"Model Accuracy: {accuracy:.2f}")

Model Accuracy: 0.62


In [71]:
# Additional step: Visualize feature importance
feature_importances = model.get_feature_importance()
feature_names = X.columns
importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importances})
importance_df = importance_df.sort_values(by='Importance', ascending=False)

print(importance_df)

                     Feature  Importance
4   discharge_disposition_id   20.379270
13                    diag_1    8.012305
12          number_inpatient    7.195950
14                    diag_2    6.232519
2                        age    5.428857
15                    diag_3    5.289500
16          number_diagnoses    4.756848
7         num_lab_procedures    4.712187
5        admission_source_id    4.605473
3          admission_type_id    3.986480
6           time_in_hospital    3.938483
9            num_medications    3.908597
8             num_procedures    3.078356
10         number_outpatient    2.546531
11          number_emergency    2.203625
25                   insulin    1.922001
0                       race    1.779003
27               diabetesMed    1.550935
18                 A1Cresult    1.468835
19                 metformin    1.255925
1                     gender    1.084707
21                 glipizide    0.917814
23              pioglitazone    0.808475
17             m

In [39]:
model.save_model("catboost_model.cbm", format="cbm")